<a href="https://colab.research.google.com/github/jorgemm83/prueba_tecnica/blob/main/Prueba_t%C3%A9cnica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
pip install requests pandas scikit-learn

In [20]:
import requests
import pandas as pd


def get_financial_data(symbol: str, interval: str):
    #API_KEY = os.getenv('API_KEY')
    API_KEY = 'POK20BXESBQBLR4J'
    URL_API = 'https://www.alphavantage.co/query'

    params = {
        'function': 'TIME_SERIES_INTRADAY',
        'symbol': symbol,
        'interval':interval,
        'outputsize': 'full',
        'apikey': API_KEY,
        'datatype': 'json'
    }


    # Realizar la solicitud GET a la API
    response = requests.get(URL_API, params)

    # Verificar si la solicitud fue exitosa (código 200)
    if response.status_code == 200:
        data = response.json()  # Convertir la respuesta a formato JSON

        #print(data)

         # Comprobar si la respuesta contiene datos válidos
    if "Time Series ("+interval+")" not in data:
        raise ValueError("Error al obtener datos de la API")

    # Convertir los datos a un DataFrame
    df = pd.DataFrame.from_dict(data['Time Series ('+interval+')'], orient='index', dtype=float)

    # Renombrar columnas para que tengan nombres más intuitivos
    df.rename(columns={
        '1. open': 'open',
        '2. high': 'high',
        '3. low': 'low',
        '4. close': 'close',
        '5. volume': 'volume'
    }, inplace=True)

    # Convertir el índice a formato de fecha
    df.index = pd.to_datetime(df.index)

    return df

# Obtener datos de Apple (AAPL)
symbol = 'IBM'
interval='5min'
financial_data = get_financial_data(symbol, interval)

# Mostrar los primeros registros
financial_data.head()

,open,high,low,close,volume
2024-09-23 19:55:00,220.40,220.4900,220.40,220.4700,278.0
2024-09-23 19:45:00,220.40,220.4000,220.25,220.2500,24.0
2024-09-23 19:35:00,220.47,220.4900,220.32,220.3200,126.0
2024-09-23 19:30:00,220.48,220.4800,220.40,220.4000,14.0
2024-09-23 19:25:00,220.40,220.4045,220.40,220.4045,7.0


In [19]:
financial_data.shape

(3199, 5)